In [1]:
from sympy import sin, cos, pi, symbols, Eq
from sympy import solve, Matrix, sqrt

# Problema 1

## (a) Parametrització 

In [2]:
x, y, z, t = symbols('x y z t', real=True)

In [26]:
eqs = [Eq(x**2 + y**2, 1), Eq(y**2 + z**2, 1)]; eqs

[x**2 + y**2 == 1, y**2 + z**2 == 1]

De la primera equació es veu que podem usar $x = \cos t$ i $y = \sin t$ per a $t\in[0,2\pi]$ com de costum.
Restant les 2 equacions obtenim que $z=\pm x$.
Alternativement es pot substituir $y$ a la segona i deduiriem també que $z = \pm\cos t$

In [4]:
r1 = {x:cos(t), y:sin(t),z: -cos(t)}
r2 = {x:cos(t), y:sin(t),z: cos(t)}

Les dues corbes parametritzades compleixen les equacions:

In [5]:
for r in (r1,r2):
    assert all(e.subs(r).simplify() for e in eqs)

## (b) Punts d'intersecció

Igualem les components de $r_1(t)=r_2(s)$ i eliminem la solució trivial $s=t$.

In [25]:
s = symbols('s', real=True)
eqi = []
for c in (x,y,z):
    e = Eq(c.subs(r1), c.subs(r2).subs(t,s))
    eqi.append(e)
eqi

[cos(t) == cos(s), sin(t) == sin(s), -cos(t) == cos(s)]

Les dues primeres equacions conjuntament forcen que $s=t$, però llavors la tercera diu $-\cos t = \cos t$.
Això només passarà si $\cos t = 0$, és a dir si $t= \pi/2, 3\pi/2$.

In [8]:
ti = (pi/2, 3*pi/2)
punt = Matrix([x,y,z])
p1 = punt.subs(r1)
p2 = punt.subs(r2)

for u in ti: 
    assert p1.subs(t,u) == p2.subs(t,u)
    print tuple(p1.subs(t,u))

(0, 1, 0)
(0, -1, 0)


## (c) Extrems dels radis curvatura

Observem que les dues corbes són isomètriques (via $z\mapsto -z$), per tant només cal estudiar-ne una.

$\vec r'(t)$ i $\vec r''(t)$ són

In [9]:
rd = p2.diff(t); print rd
rdd = rd.diff(t); print rdd

Matrix([[-sin(t)], [cos(t)], [-sin(t)]])
Matrix([[-cos(t)], [-sin(t)], [-cos(t)]])


La norma del vector tangent és (_celeritat_):

In [10]:
rd.norm()

sqrt(2*sin(t)**2 + cos(t)**2)

In [11]:
_.simplify()

sqrt(sin(t)**2 + 1)

In [12]:
rd.cross(rdd)

Matrix([
[-sin(t)**2 - cos(t)**2],
[                     0],
[ sin(t)**2 + cos(t)**2]])

In [13]:
_.norm().simplify()

sqrt(2)

In [14]:
_/rd.norm()**3

sqrt(2)/(2*sin(t)**2 + cos(t)**2)**(3/2)

In [15]:
kappa = _.simplify(); kappa

sqrt(2)/(sin(t)**2 + 1)**(3/2)

el _radi de curvatura_ és $1/\kappa$:

In [16]:
1/kappa

sqrt(2)*(sin(t)**2 + 1)**(3/2)/2

D'aquí és veu que els extrems seran quan $\sin t=0$ o $\sin t =\pm1$:

In [17]:
[1/kappa.subs(t,u) for u in (0,pi/2)]

[sqrt(2)/2, 2]

* Però també es podria derivar i igualar a zero:

In [19]:
(1/kappa).diff(t).simplify()

3*sqrt(-cos(2*t) + 3)*sin(2*t)/4

... que només s'anul·la quan $t=k\pi/2$, per a $k=0,1,2,3$.

## (d) Digueu si les corbes són planes

* Una manera directa de fer-ho és observar que $r_1$ és dins del pla $x + z=0$ i que $r_2$ és dins de $x -z = 0$.

* Una altra manera (molt més llarga) és calcular-ne la _torsió_ i comprovar que doni zero.

In [20]:
rddd = rdd.diff(t); rddd

Matrix([
[ sin(t)],
[-cos(t)],
[ sin(t)]])

In [21]:
Matrix([u.transpose() for u in (rd,rdd,rdd)])

Matrix([
[-sin(t),  cos(t), -sin(t)],
[-cos(t), -sin(t), -cos(t)],
[-cos(t), -sin(t), -cos(t)]])

In [22]:
_.det()

0

Aquest determinant (que és igual a $(\vec r'\times\vec r'')\cdot \vec r'''$) és el que surt al numerador de la fórmula de $\tau$. Per tant la corba és plana.

## (e) Radi de la bola més grossa

### Per Lagrange

Es tracta de trobar quina és la distància mínima a l'origen de coordenades d'un punt de la intersecció dels dos cilindres.
Una bola centrada a l'origen amb radi més gran contindrà necessàriament aquest punt i, per tant, ja no serà vàlida.

Plantegem la _Laplaciana_ amb dues condicions:
$$\lambda \left(x^{2} + y^{2} - 1\right) + \mu \left(y^{2} + z^{2} - 1\right) + x^{2} + y^{2} + z^{2}$$

In [27]:
l,m = symbols('lambda mu')
from sympy import latex

L = x**2 + y**2 + z**2 + sum(g*(e.lhs - e.rhs) for g,e in zip((l,m), eqs)); L

lambda*(x**2 + y**2 - 1) + mu*(y**2 + z**2 - 1) + x**2 + y**2 + z**2

El sistema que s'ha de resoldre és $\nabla L = \vec0$:

In [39]:
sols = []
leqs = [Eq(L.diff(v).factor()) for v in (x,y,z,l,m)]; leqs

[2*x*(lambda + 1) == 0,
 2*y*(lambda + mu + 1) == 0,
 2*z*(mu + 1) == 0,
 x**2 + y**2 - 1 == 0,
 y**2 + z**2 - 1 == 0]

la tercera equació ens diu que $z=0$ o $\mu = -1$.

Anem cas per cas:

* Cas $z=0$:

In [40]:
sol = {z:0}
ez = [e.subs(sol) for e in leqs if e.subs(sol) != True]; ez

[2*x*(lambda + 1) == 0,
 2*y*(lambda + mu + 1) == 0,
 x**2 + y**2 - 1 == 0,
 y**2 - 1 == 0]

Resolem per a $y$ a la darrera equació:

In [41]:
ys = solve(ez[-1],y, dict=True); ys

[{y: -1}, {y: 1}]

In [42]:
for u in ys: 
    sol.update(u)
    print sol,[e.subs(sol) for e in ez if e.subs(sol) != True]

{y: -1, z: 0} [2*x*(lambda + 1) == 0, -2*lambda - 2*mu - 2 == 0, x**2 == 0]
{y: 1, z: 0} [2*x*(lambda + 1) == 0, 2*lambda + 2*mu + 2 == 0, x**2 == 0]


Les dues alternatives forcen que $x=0$ a la darrera equació:

In [43]:
sol[x] = 0
for u in ys: 
    sol.update(u)
    sols.append(dict(sol))
    print tuple(punt.subs(sol))
    print [e.subs(sol) for e in ez if e.subs(sol) != True]

(0, -1, 0)
[-2*lambda - 2*mu - 2 == 0]
(0, 1, 0)
[2*lambda + 2*mu + 2 == 0]


i les dues alternatives són possibles. Això ens dóna 2 punts.

* Cas $\mu = -1$:

In [44]:
sol = {m: -1}
em = [e.subs(sol) for e in leqs if e.subs(sol) != True]; em

[2*x*(lambda + 1) == 0,
 2*lambda*y == 0,
 x**2 + y**2 - 1 == 0,
 y**2 + z**2 - 1 == 0]

Observant les dues primeres veiem que $x=0$ o $\lambda=-1$ combinat amb $\lambda=0$ o $y=0$.
Examinant les alternatives de $\lambda$:

* Cas $\lambda=0$, $\mu = -1$:

In [45]:
sol.update({l: 0})
[e.subs(sol) for e in leqs if e.subs(sol) != True]

[2*x == 0, x**2 + y**2 - 1 == 0, y**2 + z**2 - 1 == 0]

d'aquí es deduix que $x=0$:

In [46]:
sol.update({x: 0})
set(e.subs(sol) for e in leqs if e.subs(sol) != True)

{y**2 - 1 == 0, y**2 + z**2 - 1 == 0}

Ara resulta que $y^2=1$, amb la qual cosa $z=0$:

In [47]:
sol.update({z:0})
set(e.subs(sol) for e in leqs if e.subs(sol) != True)

{y**2 - 1 == 0}

In [48]:
solve(_, dict=True)

[{y: -1}, {y: 1}]

i obtenim els punts següents (que, de fet, ja havien sortit):

In [49]:
for u in _:
    sol.update(u)
    print tuple(punt.subs(sol))
    sols.append(dict(sol))

(0, -1, 0)
(0, 1, 0)


* Cas $\lambda = \mu = -1$:

In [50]:
sol = {m: -1, l:-1}
set(e.subs(sol) for e in leqs if e.subs(sol) != True)

{-2*y == 0, y**2 + z**2 - 1 == 0, x**2 + y**2 - 1 == 0}

Ara veiem que per força $y=0$:

In [51]:
sol[y] = 0
set(e.subs(sol) for e in leqs if e.subs(sol) != True)

{x**2 - 1 == 0, z**2 - 1 == 0}

In [52]:
solve(_, dict=True)

[{x: -1, z: -1}, {x: -1, z: 1}, {x: 1, z: -1}, {x: 1, z: 1}]

Això ens porta a $x=\pm1$, $z=\pm1$ i obtenim els punts:

In [53]:
for u in _:
    sol.update(u)
    print tuple(punt.subs(sol))
    sols.append(dict(sol))

(-1, 0, -1)
(-1, 0, 1)
(1, 0, -1)
(1, 0, 1)


ja hem mirat totes les combinacions.
Ara el que queda és avaluar la distància a cada punt possible i quedar-nos amb el valor més petit:

In [54]:
f = punt.norm(); f

sqrt(x**2 + y**2 + z**2)

In [55]:
for s in sols:
    d = f.subs(s)
    print d, tuple(punt.subs(s))

1 (0, -1, 0)
1 (0, 1, 0)
1 (0, -1, 0)
1 (0, 1, 0)
sqrt(2) (-1, 0, -1)
sqrt(2) (-1, 0, 1)
sqrt(2) (1, 0, -1)
sqrt(2) (1, 0, 1)


### Directament

Aquesta alternativa no és legal ja que es demanava que és fes per _Lagrange_ però correcta:

In [57]:
d2 = punt.subs(r1).norm()**2; d2

sin(t)**2 + 2*cos(t)**2

In [58]:
d2.diff(t)

-2*sin(t)*cos(t)

In [59]:
for u in solve(_,t):
    print u, tuple(punt.subs(r1).subs(t,u)), sqrt(d2.subs(t,u))

0 (1, 0, -1) sqrt(2)
pi/2 (0, 1, 0) 1
pi (-1, 0, 1) sqrt(2)
3*pi/2 (0, -1, 0) 1


# Problema 2

In [393]:
x,y,z = symbols('x y z', real=True)
a = symbols('a', real=True)
r = symbols('r', positive=True)
l = symbols('lambda', real=True)

f = 6 + x**2 - y**2 + 2*x*y

## (a) Altura de l'edifici

El problema és calcular el màxim de $f(x,y) = 6 + x^2 - y^2 + 2xy$ dins del compacte $R = \{(x,y)\mid x^2 + y^2 \leq4\}$. 

Anem pas a pas:

1. Estudiem els posibles extrems a l'interior de $R$

2. Estudiem els possibles extrems a la frontera de $R$.
   Aquest segon cas es pot fer de dues maneres que detallarem d'aquí a un mooment.
   
   
### Estudi de l'interior

Busquem els _punts crítics_ imposant $\nabla f = \vec 0$:

In [394]:
[diff(f,v) for v in (x,y)]

[2*x + 2*y, 2*x - 2*y]

això correspon a un sistema lineal amb solució única:

In [395]:
sols = []
[s] = solve(_,dict=True); print s
sols.append(s)

{y: 0, x: 0}


### Estudi de la frontera

El problema a la frontera està condicinat per $x^2 + y^2 = 4$ i com deiem es pot fer de dues maneres:

#### Pel mètode dels _multiplicadors de Lagrange_

Posant com a _Lagrangiana_: $$L = - \lambda \left(x^{2} + y^{2} - 4\right) + x^{2} + 2 x y - y^{2} + 6$$

In [396]:
L = f - l*(x**2 + y**2 - 4)
eqs = [diff(L,c).factor() for c in (x,y,l)]
eqs

[-2*(lambda*x - x - y), 2*(-lambda*y + x - y), -x**2 - y**2 + 4]

In [397]:
Matrix([[diff(c,v) for v in (x,y)] for c in eqs[:2]])

Matrix([
[-2*lambda + 2,             2],
[            2, -2*lambda - 2]])

In [398]:
_.det()

4*lambda**2 - 8

In [399]:
solve(_, dict=True)

[{lambda: -sqrt(2)}, {lambda: sqrt(2)}]

Com abans procedim cas per cas:

* $\lambda=\pm\sqrt2$


   En aquest cas el sistema lineal de les 2 primeres equacions és indeterminat i podem pendre qualsevol de les equacions:

In [400]:
for u in _:
    [s] = solve(eqs[0].subs(u), dict=True)
    u.update(s)
    e = eqs[-1].subs(u)
    for t in solve(e,y,dict=True):
        u.update(t)
        assert all(e.subs(u).simplify()==0 for e in eqs)
        print u
        sols.append(u)

{y: -sqrt(2)/sqrt(-sqrt(2) + 2), x: -sqrt(2)*y + y, lambda: -sqrt(2)}
{y: sqrt(2)/sqrt(-sqrt(2) + 2), x: -sqrt(2)*y + y, lambda: -sqrt(2)}
{y: -sqrt(2)/sqrt(sqrt(2) + 2), x: y + sqrt(2)*y, lambda: sqrt(2)}
{y: sqrt(2)/sqrt(sqrt(2) + 2), x: y + sqrt(2)*y, lambda: sqrt(2)}


* Cas $\lambda^2 \neq2$

   En aquest cas el sistema linieal de dalt només té la solució $x=y=0$, però això no compleix la darrera equació:

In [401]:
sol = {x:0, y:0}
[e.subs(sol) for e in eqs]

[0, 0, 4]

In [402]:
for s in sols:
    p = Matrix([x,y]).subs(s)
    p.simplify()
    zval = f.subs(s).simplify()
    print tuple(p), zval, float(zval)

(0, 0) 6 6.0
(sqrt(-sqrt(2) + 2), sqrt(2)/sqrt(-sqrt(2) + 2)) -4*sqrt(2) + 6 0.343145750508
(sqrt(-sqrt(2) + 2), sqrt(2)/sqrt(-sqrt(2) + 2)) -4*sqrt(2) + 6 0.343145750508
(sqrt(sqrt(2) + 2), sqrt(2)/sqrt(sqrt(2) + 2)) 4*sqrt(2) + 6 11.6568542495
(sqrt(sqrt(2) + 2), sqrt(2)/sqrt(sqrt(2) + 2)) 4*sqrt(2) + 6 11.6568542495


#### Parametritzant la frontera

Com que la condició $x^2+y^2=4$ correspon a una circumferéncia de radi 2, podem parametrizar-ho com una corba:

In [415]:
rt = {x: 2*cos(a), y: 2*sin(a)}
h = f.subs(rt).simplify()
h

4*sqrt(2)*sin(2*a + pi/4) + 6

In [416]:
h.diff(a)

8*sqrt(2)*cos(2*a + pi/4)

In [417]:
solve(_,a)

[pi/8, 5*pi/8]

In [418]:
_ + [c + pi for c in _ if c+pi < 2*pi]

[pi/8, 5*pi/8, 9*pi/8, 13*pi/8]

In [419]:
for u in _:
    print tuple(Matrix([x,y]).subs(S).subs(a,u)), float(f.subs(S).subs(a,u))

(2*sqrt(sqrt(2)/4 + 1/2), 2*sqrt(-sqrt(2)/4 + 1/2)) 11.6568542495
(-2*sqrt(-sqrt(2)/4 + 1/2), 2*sqrt(sqrt(2)/4 + 1/2)) 0.343145750508
(-2*sqrt(sqrt(2)/4 + 1/2), -2*sqrt(-sqrt(2)/4 + 1/2)) 11.6568542495
(2*sqrt(-sqrt(2)/4 + 1/2), -2*sqrt(sqrt(2)/4 + 1/2)) 0.343145750508


#### Quina forma és?

In [420]:
w = symbols('w', real=True)
fHom = ((f-z).subs({x: x/w, y:y/w, z:z/w})*w**2).expand(); fHom

6*w**2 - w*z + x**2 + 2*x*y - y**2

In [421]:
fq = Matrix([[c.diff(v)/2 for v in (x,y,z,w)] for c in [fHom.diff(u) for u in (x,y,z,w)]] )
print fq

Matrix([[1, 1, 0, 0], [1, -1, 0, 0], [0, 0, 0, -1/2], [0, 0, -1/2, 6]])


In [422]:
for l,g,vs in fq.eigenvects():
    print l
    assert g == 1
    print vs[0]

-sqrt(2)
Matrix([[-1/(1 + sqrt(2))], [1], [0], [0]])
sqrt(2)
Matrix([[-1/(-sqrt(2) + 1)], [1], [0], [0]])
3 + sqrt(37)/2
Matrix([[0], [0], [1/(2*(-sqrt(37)/2 - 3))], [1]])
-sqrt(37)/2 + 3
Matrix([[0], [0], [1/(2*(-3 + sqrt(37)/2))], [1]])


## (b) Àrea de la coberta

In [423]:
from sympy import integrate
R ={x: r*cos(a), y: r*sin(a), z: f}
R.update({z: R[z].subs(R).simplify()})
R

{y: r*sin(a), x: r*cos(a), z: sqrt(2)*r**2*sin(2*a + pi/4) + 6}

In [426]:
Rr = Matrix([diff(c.subs(R), r) for c in (x,y,z)]); print Rr
Ra = Matrix([diff(c.subs(R), a) for c in (x,y,z)]); print Ra

Matrix([[cos(a)], [sin(a)], [2*sqrt(2)*r*sin(2*a + pi/4)]])
Matrix([[-r*sin(a)], [r*cos(a)], [2*sqrt(2)*r**2*cos(2*a + pi/4)]])


In [427]:
from sympy import powdenest
dR = powdenest(Rr.cross(Ra)).simplify(); print dR
dR.norm().simplify()

Matrix([
[-2*sqrt(2)*r**2*sin(a + pi/4)],
[-2*sqrt(2)*r**2*cos(a + pi/4)],
[                            r]])


r*sqrt(8*r**2 + 1)

In [428]:
integrate(integrate(_, (r,0,2)), (a,0,2*pi))

2*pi*(-1/24 + 11*sqrt(33)/8)

In [429]:
float(_)

49.36765908543893

#### Amb rectangulars

In [430]:
Matrix([-diff(f,x), -diff(f,y), 1])

Matrix([
[-2*x - 2*y],
[-2*x + 2*y],
[         1]])

In [431]:
_.norm().simplify()

sqrt(8*x**2 + 8*y**2 + 1)

In [432]:
_.subs({x: r*cos(a), y: r*sin(a)}).simplify()

sqrt(8*r**2 + 1)

In [433]:
integrate(integrate(_*r, (r,0,2)), (a,0,2*pi))

2*pi*(-1/24 + 11*sqrt(33)/8)

## (c) Àrea lateral

In [443]:
S = {x: 2*cos(a), y:2*sin(a)}
Sa = Matrix([diff(c.subs(S), a) for c in (x,y,z)]); print Sa
Sz = Matrix([diff(c.subs(S), z) for c in (x,y,z)]); print Sz

Matrix([[-2*sin(a)], [2*cos(a)], [0]])
Matrix([[0], [0], [1]])


In [444]:
dS = Sa.cross(Sz); print dS
dSn = dS.norm().simplify(); dSn

Matrix([[2*cos(a)], [2*sin(a)], [0]])


2

In [445]:
zmax = f.subs(S); zmax.simplify()

4*sqrt(2)*sin(2*a + pi/4) + 6

In [446]:
integrate(dSn, (z,0,zmax))

-8*sin(a)**2 + 16*sin(a)*cos(a) + 8*cos(a)**2 + 12

In [447]:
areaL = integrate(_, (a,0,2*pi)); areaL

24*pi

## (d) Flux de la coberta

### Directament

In [450]:
F = Matrix([x,y,z])

In [451]:
F.subs(R)

Matrix([
[                        r*cos(a)],
[                        r*sin(a)],
[sqrt(2)*r**2*sin(2*a + pi/4) + 6]])

In [452]:
dR.simplify()

Matrix([
[-2*sqrt(2)*r**2*sin(a + pi/4)],
[-2*sqrt(2)*r**2*cos(a + pi/4)],
[                            r]])

In [453]:
F.dot(dR).simplify()

r*(-2*sqrt(2)*r*x*sin(a + pi/4) - 2*sqrt(2)*r*y*cos(a + pi/4) + z)

In [454]:
integrate(_, (r,0,2))

-16*sqrt(2)*x*sin(a + pi/4)/3 - 16*sqrt(2)*y*cos(a + pi/4)/3 + 2*z

In [455]:
integrate(_, (a,0,2*pi))

4*pi*z

In [456]:
F.subs(R)

Matrix([
[                        r*cos(a)],
[                        r*sin(a)],
[sqrt(2)*r**2*sin(2*a + pi/4) + 6]])

In [457]:
_.dot(dR)

-2*sqrt(2)*r**3*sin(a)*cos(a + pi/4) - 2*sqrt(2)*r**3*sin(a + pi/4)*cos(a) + r*(sqrt(2)*r**2*sin(2*a + pi/4) + 6)

In [458]:
_.simplify()

r*(-sqrt(2)*r**2*sin(2*a + pi/4) + 6)

In [459]:
integrate(_, (r,0,2))

-4*sqrt(2)*sin(2*a + pi/4) + 12

In [460]:
integrate(_, (a,0,2*pi))

24*pi

#### Amb cartesianes

In [461]:
XY = F.subs(z,f); XY

Matrix([
[                      x],
[                      y],
[x**2 + 2*x*y - y**2 + 6]])

In [462]:
XYx = Matrix([diff(c, x) for c in XY]); print XYx
XYy = Matrix([diff(c, y) for c in XY]); print XYy
XYx.cross(XYy)

Matrix([[1], [0], [2*x + 2*y]])
Matrix([[0], [1], [2*x - 2*y]])


Matrix([
[-2*x - 2*y],
[-2*x + 2*y],
[         1]])

In [463]:
F.subs(z,f).dot(_).simplify()

-x**2 - 2*x*y + y**2 + 6

In [464]:
(_.subs({x: r*cos(a), y: r*sin(a)})*r)

r*(r**2*sin(a)**2 - 2*r**2*sin(a)*cos(a) - r**2*cos(a)**2 + 6)

In [465]:
integrate(_, (r,0,2))

4*sin(a)**2 - 8*sin(a)*cos(a) - 4*cos(a)**2 + 12

In [466]:
integrate(_, (a,0,2*pi))

24*pi

### Usant el teorema de la divergència

#### Part de la divergència

In [468]:
divF = sum(diff(c,u) for c,u in zip(F,[x,y,z])); divF

3

In [469]:
integrate(divF, (z,0,f))

3*x**2 + 6*x*y - 3*y**2 + 18

In [470]:
_.subs({x: r*cos(a), y:r*sin(a)}).simplify()

3*sqrt(2)*r**2*sin(2*a + pi/4) + 18

In [471]:
integrate(_*r, (r,0,2))

12*sqrt(2)*sin(2*a + pi/4) + 36

In [472]:
Fdiv = integrate(_, (a,0,2*pi)); Fdiv

72*pi

#### Part de l'area lateral

In [476]:
print S
print dS
F.subs(S).dot(dS).simplify()

{y: 2*sin(a), x: 2*cos(a)}
Matrix([[2*cos(a)], [2*sin(a)], [0]])


4

In [477]:
integrate(_, (z,0,f.subs(S)))

-16*sin(a)**2 + 32*sin(a)*cos(a) + 16*cos(a)**2 + 24

In [478]:
FL = integrate(_, (a,0,2*pi)); FL

48*pi

#### Tapa inferior

In [479]:
T = {x: r*cos(a), y: r*sin(a), z: 0}
Tr = Matrix([diff(c.subs(T),r) for c in (x,y,z)]); print Tr
Ta = Matrix([diff(c.subs(T),a) for c in (x,y,z)]); print Ta
dT = Tr.cross(Ta)
dT.simplify(); print dT
FI = F.subs(T).dot(dT); FI

Matrix([[cos(a)], [sin(a)], [0]])
Matrix([[-r*sin(a)], [r*cos(a)], [0]])
Matrix([[0], [0], [r]])


0

In [480]:
Fdiv - FL + FI

24*pi

In [ ]:
from sympy.plotting import plot3d_parametric_surface

In [ ]:
w = symbols('w', real=True)

In [ ]:
pS = plot3d_parametric_surface(x.subs(R), y.subs(R), z.subs(R), (r,0,2), (a,0,2*pi))

In [ ]:
pL = plot3d_parametric_surface(x.subs(S), y.subs(S), w*f.subs(S), (a,0,2*pi), (w,0,1))

In [ ]:
pS.extend(pL)

In [ ]:
pS.show()

In [ ]:
z